In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trackml-particle-identification/train_sample.zip
/kaggle/input/trackml-particle-identification/train_3.zip
/kaggle/input/trackml-particle-identification/train_2.zip
/kaggle/input/trackml-particle-identification/sample_submission.csv.zip
/kaggle/input/trackml-particle-identification/train_5.zip
/kaggle/input/trackml-particle-identification/detectors.zip
/kaggle/input/trackml-particle-identification/test.zip
/kaggle/input/trackml-particle-identification/blacklist_training.zip
/kaggle/input/trackml-particle-identification/train_4.zip
/kaggle/input/trackml-particle-identification/train_1.zip


In [2]:
!pip install torch_geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.8.0+cu126.html


Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.transforms import KNNGraph
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv
import numpy as np
import os
import pandas as pd
import pytorch_lightning as pl
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import zipfile
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
import gc

In [5]:
torch.cuda.empty_cache()
gc.collect()
print("✅ GPU memory cleared")

✅ GPU memory cleared


In [7]:

# Check available GPU memory
if torch.cuda.is_available():
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"📊 Total GPU memory: {total_memory:.2f} GB")
    print(f"📊 Available GPU memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB allocated")


📊 Total GPU memory: 14.74 GB
📊 Available GPU memory: 0.00 GB allocated


In [8]:
# Data exploration utility
def explore_data_files(root):
    """Explore the structure of data files"""
    print("="*60)
    print("DATA FILE EXPLORATION")
    print("="*60)
    
    data_files = {
        'train_sample': 'train_sample.zip',
        'train_1': 'train_1.zip',
        'train_2': 'train_2.zip',
        'train_3': 'train_3.zip',
        'train_4': 'train_4.zip',
        'train_5': 'train_5.zip',
        'test': 'test.zip',
    }
    
    for name, filename in data_files.items():
        filepath = os.path.join(root, filename)
        if os.path.exists(filepath):
            with zipfile.ZipFile(filepath, 'r') as z:
                files = z.namelist()
                events = len([f for f in files if f.endswith('-hits.csv')])
                print(f"\n{name:15} ({filename:20}): {events:4} events")
        else:
            print(f"\n{name:15} ({filename:20}): NOT FOUND")
    
    print("\n" + "="*60)

In [9]:
# Explore data before training
root = '/kaggle/input/trackml-particle-identification'
explore_data_files(root)

print("\n" + "="*60)
print("⚡ QUICK TEST MODE ACTIVATED ⚡")
print("="*60)
print("This configuration uses minimal data for rapid testing:")
print("  • Training: 5 events (~2-3 min)")
print("  • Validation: 3 events")
print("  • Testing: 3 events")
print("  • Epochs: 5")
print("\nPerfect for:")
print("  ✓ Verifying code runs without errors")
print("  ✓ Quick model architecture testing")
print("  ✓ Debugging and iteration")
print("\n💡 For full training, modify these values:")
print("  • train_dataset: n_events=50+ from train_1.zip")
print("  • val_dataset: n_events=20+ from train_sample.zip")
print("  • test_dataset: n_events=None (all 125 events)")
print("  • trainer: max_epochs=20-30")
print("="*60 + "\n")

DATA FILE EXPLORATION

train_sample    (train_sample.zip    ):  100 events

train_1         (train_1.zip         ): 1770 events

train_2         (train_2.zip         ): 1770 events

train_3         (train_3.zip         ): 1770 events

train_4         (train_4.zip         ): 1770 events

train_5         (train_5.zip         ): 1770 events

test            (test.zip            ):  125 events


⚡ QUICK TEST MODE ACTIVATED ⚡
This configuration uses minimal data for rapid testing:
  • Training: 5 events (~2-3 min)
  • Validation: 3 events
  • Testing: 3 events
  • Epochs: 5

Perfect for:
  ✓ Verifying code runs without errors
  ✓ Quick model architecture testing
  ✓ Debugging and iteration

💡 For full training, modify these values:
  • train_dataset: n_events=50+ from train_1.zip
  • val_dataset: n_events=20+ from train_sample.zip
  • test_dataset: n_events=None (all 125 events)
  • trainer: max_epochs=20-30



In [10]:
# Manual data loading from zip
def load_event(zip_path, inner_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        hits = pd.read_csv(z.open(f'{inner_path}-hits.csv'))
        cells = pd.read_csv(z.open(f'{inner_path}-cells.csv'))
        particles = pd.read_csv(z.open(f'{inner_path}-particles.csv'))
        truth = pd.read_csv(z.open(f'{inner_path}-truth.csv'))
    return hits, cells, particles, truth


In [11]:
# Dataset class adjusted for Kaggle zip structure with memory-efficient sampling
class TrackMLDataset(Dataset):
    def __init__(self, root, zip_name='train_sample.zip', n_events=8, transform=None, max_hits=30000):
        super().__init__(root, transform=transform)
        self.zip_path = os.path.join(root, zip_name)
        self.max_hits = max_hits  # Configurable hit limit
        # Reduce k from 8 to 6 to save memory
        self.transform = transform or KNNGraph(k=6)
        
        # Dynamically extract unique event paths from zip namelist
        self.inner_paths = []
        with zipfile.ZipFile(self.zip_path, 'r') as z:
            for fname in z.namelist():
                if fname.endswith('-hits.csv'):
                    inner = fname.rsplit('-hits.csv', 1)[0]
                    self.inner_paths.append(inner)
        self.inner_paths = sorted(set(self.inner_paths))[:n_events]

    def len(self):
        return len(self.inner_paths)

    def get(self, idx):
        inner_path = self.inner_paths[idx]
        hits, cells, particles, truth = load_event(self.zip_path, inner_path)
        
        # MEMORY OPTIMIZATION: Sample hits if too many
        if len(hits) > self.max_hits:
            sample_idx = np.random.choice(len(hits), self.max_hits, replace=False)
            sample_idx = np.sort(sample_idx)  # Keep order for consistency
            hits = hits.iloc[sample_idx].reset_index(drop=True)
            # Update truth to match sampled hits
            truth = truth[truth['hit_id'].isin(hits['hit_id'])].reset_index(drop=True)
        
        # Node features (r, phi, z)
        r = np.sqrt(hits['x']**2 + hits['y']**2).values
        phi = np.arctan2(hits['y'], hits['x']).values
        z = hits['z'].values
        x = torch.tensor(np.stack([r, phi, z], axis=1), dtype=torch.float)
        
        # Store global coordinates for KNN calculation
        pos = torch.tensor(hits[['x', 'y', 'z']].values, dtype=torch.float)
        
        data = Data(x=x, pos=pos)
        
        # Apply KNN Transform to create edges
        if self.transform:
            data = self.transform(data)
            
            # Label the edges: 1 if both hits belong to the same particle, else 0
            hit_to_particle = truth.set_index('hit_id')['particle_id'].to_dict()
            
            raw_hit_ids = hits['hit_id'].values
            edge_src_pids = np.array([hit_to_particle.get(raw_hit_ids[i], 0) for i in data.edge_index[0]])
            edge_tgt_pids = np.array([hit_to_particle.get(raw_hit_ids[i], 0) for i in data.edge_index[1]])
            
            # Logic: same particle AND not noise (PID 0 is noise)
            y = (edge_src_pids == edge_tgt_pids) & (edge_src_pids != 0)
            data.y = torch.tensor(y, dtype=torch.float)
            
        return data

In [12]:
# Model - Smaller architecture for memory efficiency
class TrackTransformer(torch.nn.Module):
    def __init__(self, in_channels=3, hidden_channels=32, out_channels=1, heads=2):  # Reduced from 64/4 to 32/2
        super().__init__()
        self.hidden_channels = hidden_channels
        self.heads = heads
        
        self.embed = torch.nn.Linear(in_channels, hidden_channels)
        
        # Edge dimension matches the OUTPUT of each layer
        self.conv1 = TransformerConv(hidden_channels, hidden_channels, heads=heads, 
                                     dropout=0.1, edge_dim=hidden_channels, beta=True)
        # After conv1: output is hidden_channels * heads = 64
        self.conv2 = TransformerConv(hidden_channels * heads, hidden_channels, heads=heads, 
                                     dropout=0.1, edge_dim=hidden_channels * heads, beta=True)
        # After conv2: output is hidden_channels * heads = 64
        self.conv3 = TransformerConv(hidden_channels * heads, hidden_channels // heads, heads=heads, 
                                     dropout=0.1, edge_dim=hidden_channels * heads, beta=True)
        
        # Final edge classifier
        self.lin_edge = torch.nn.Linear((hidden_channels // heads) * heads * 2, out_channels)

    def forward(self, x, edge_index, batch=None):
        x = F.relu(self.embed(x))
        
        # Layer 1: edge_attr size = 32
        src, tgt = x[edge_index[0]], x[edge_index[1]]
        edge_attr = torch.abs(src - tgt)
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        
        # Layer 2: edge_attr size = 64 (32 * 2 heads)
        src, tgt = x[edge_index[0]], x[edge_index[1]]
        edge_attr = torch.abs(src - tgt)
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        
        # Layer 3: edge_attr size = 64 (32 * 2 heads)
        src, tgt = x[edge_index[0]], x[edge_index[1]]
        edge_attr = torch.abs(src - tgt)
        x = self.conv3(x, edge_index, edge_attr)
        
        # Concatenate source and target node embeddings for edge classification
        edge_emb = torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1)
        return self.lin_edge(edge_emb).squeeze(-1)

In [34]:
class TrackLightning(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        
        # Storage for validation predictions
        self.validation_step_outputs = []

    def forward(self, batch):
        return self.model(batch.x, batch.edge_index, batch.batch)

    def training_step(self, batch, batch_idx):
        logits = self(batch)
        # Ensure target is float for BCEWithLogitsLoss
        loss = self.loss_fn(logits, batch.y.float())
        
        with torch.no_grad():
            preds = torch.sigmoid(logits) > 0.5
            acc = (preds == batch.y).float().mean()
        
        self.log('train_loss', loss, prog_bar=True, batch_size=batch.num_graphs)
        self.log('train_acc', acc, prog_bar=True, batch_size=batch.num_graphs)
        
        return loss
    
    def on_train_batch_end(self, outputs, batch, batch_idx):
        # Memory management: manually clear cache if needed
        if batch_idx % 5 == 0: 
            torch.cuda.empty_cache()

    def validation_step(self, batch, batch_idx):
        logits = self(batch)
        loss = self.loss_fn(logits, batch.y.float())
        
        preds = (torch.sigmoid(logits) > 0.5).cpu()
        targets = batch.y.cpu()
        
        # Store as a dictionary to keep things organized
        output = {'preds': preds, 'targets': targets}
        self.validation_step_outputs.append(output)
        
        acc = (preds == targets).float().mean()
        self.log('val_loss', loss, prog_bar=True, batch_size=batch.num_graphs)
        self.log('val_acc', acc, prog_bar=True, batch_size=batch.num_graphs)
        
        return loss
    
    def on_validation_epoch_end(self):
        # Concatenate all stored predictions and targets
        all_preds = torch.cat([x['preds'] for x in self.validation_step_outputs]).numpy()
        all_targets = torch.cat([x['targets'] for x in self.validation_step_outputs]).numpy()
        
        # Calculate metrics
        accuracy = accuracy_score(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, zero_division=0)
        recall = recall_score(all_targets, all_preds, zero_division=0)
        f1 = f1_score(all_targets, all_preds, zero_division=0)
        
        # Log metrics - these will show up in your progress bar and logs 
        # instead of causing a print recursion crash
        self.log('val_accuracy', accuracy, prog_bar=True)
        self.log('val_precision', precision)
        self.log('val_recall', recall)
        self.log('val_f1', f1, prog_bar=True)
        
        # CRITICAL: Clear storage to free RAM and prevent RecursionError
        self.validation_step_outputs.clear()
        torch.cuda.empty_cache()

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=1e-3, weight_decay=1e-5)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        return {
            'optimizer': optimizer, 
            'lr_scheduler': {
                'scheduler': scheduler, 
                'monitor': 'val_loss'
            }
        }
    def on_test_epoch_end(self):
        return self.on_validation_epoch_end()

In [23]:
# Prepare data with train/val split using multiple training files
root = '/kaggle/input/trackml-particle-identification'

In [24]:
# Load training data with more events (can handle more with batching!)
print("\nLoading training data...")
train_dataset = TrackMLDataset(root, zip_name='train_sample.zip', n_events=8)  # More events with batching!
print(f"  Training events: {len(train_dataset)}")


Loading training data...
  Training events: 8


In [25]:
# Load validation data
print("Loading validation data...")
val_dataset = TrackMLDataset(root, zip_name='train_sample.zip', n_events=4)
print(f"  Validation events: {len(val_dataset)}")

full_train_dataset = train_dataset

print(f"\n{'='*60}")
print(f"Dataset Summary:")
print(f"  Training:   {len(full_train_dataset)} events")
print(f"  Validation: {len(val_dataset)} events")
print(f"  Batch size: 2 events per batch (uses less memory)")
print(f"  Estimated training time: ~5-8 minutes")
print(f"{'='*60}\n")

Loading validation data...
  Validation events: 4

Dataset Summary:
  Training:   8 events
  Validation: 4 events
  Batch size: 2 events per batch (uses less memory)
  Estimated training time: ~5-8 minutes



In [26]:
BATCH_SIZE = 2  # Process 2 events at once

train_loader = DataLoader(
    full_train_dataset, 
    batch_size=BATCH_SIZE,  # Batch multiple graphs together!
    shuffle=True, 
    num_workers=0,
    pin_memory=True  # Faster GPU transfer
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE,  # Same batch size for validation
    shuffle=False, 
    num_workers=0,
    pin_memory=True
)

print(f"✅ DataLoaders created:")
print(f"   Training batches: {len(train_loader)} (batch_size={BATCH_SIZE})")
print(f"   Validation batches: {len(val_loader)} (batch_size={BATCH_SIZE})")
print(f"   Total training graphs: {len(full_train_dataset)}")
print(f"   Total validation graphs: {len(val_dataset)}\n")

✅ DataLoaders created:
   Training batches: 4 (batch_size=2)
   Validation batches: 2 (batch_size=2)
   Total training graphs: 8
   Total validation graphs: 4



In [27]:
# Train
model = TrackTransformer()
lightning_model = TrackLightning(model)

In [28]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# 1. Setup Callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    dirpath='checkpoints',
    filename='trackml-{epoch:02d}-{val_f1:.4f}',
    save_top_k=3,
    mode='max'
)

early_stop_callback = EarlyStopping(
    monitor='val_f1', # Changed to monitor F1 as it's better for TrackML
    patience=5,
    mode='max'
)

# 2. Initialize Trainer with Memory Optimizations
trainer = pl.Trainer(
    max_epochs=5,
    accelerator='gpu',
    devices=1,
    precision='16-mixed',       # Reduces VRAM usage by ~50%
    accumulate_grad_batches=2,  # Simulates larger batch size
    gradient_clip_val=1.0,      # Improves training stability
    callbacks=[checkpoint_callback, early_stop_callback],
    num_sanity_val_steps=1,     # Quick check before full training
    log_every_n_steps=10        # Reduces console/logger clutter
)

# 3. Start Training
print("🚀 Initializing training on GPU...")
trainer.fit(lightning_model, train_loader, val_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


🚀 Initializing training on GPU...


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:242: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model   │ TrackTransformer  │ 42.3 K │ train │     0 │
│ 1 │ loss_fn │ BCEWithLogitsLoss │      0 │ train │     0 │
└───┴─────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 42.3 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 42.3 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 28                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

...

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 
'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 
'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:317: The number of training batches (4)
is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

==================================================

`Trainer.fit` stopped: `max_epochs=5` reached.


In [36]:
# 1. Initialize and Load (Same as before)
base_model = TrackTransformer(in_channels=3, hidden_channels=32, out_channels=1, heads=2)
model_eval = TrackLightning.load_from_checkpoint(
    'checkpoints/trackml-epoch=02-val_f1=0.0528.ckpt', 
    model=base_model
)

# 2. Use validate() instead of test() to avoid the error
trainer.validate(model_eval, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.5498277544975281     │
│       val_accuracy        │    0.5498277544975281     │
│          val_f1           │    0.05163121223449707    │
│         val_loss          │            nan            │
│       val_precision       │   0.026981733739376068    │
│        val_recall         │    0.5973190665245056     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': nan,
  'val_acc': 0.5498277544975281,
  'val_accuracy': 0.5498277544975281,
  'val_precision': 0.026981733739376068,
  'val_recall': 0.5973190665245056,
  'val_f1': 0.05163121223449707}]